# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [26]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, HashingVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.externals import joblib

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', engine)  
X = df["message"]
Y = df.iloc[:,-36:]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    tokenizer = RegexpTokenizer(r'\w+')
    lemmatizer = WordNetLemmatizer()

    tokens = tokenizer.tokenize(text)

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
'''
This model uses the following steps recommended in 
https://towardsdatascience.com/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a 
for multi-output classification task:
1. CountVectorizer()
2. TfidfTransformer()
3. Classifier

'''

pipeline =  Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(DecisionTreeClassifier()))
                ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [7]:
# Split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=21)

# Train pipeline
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ion_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
# Infer test data with trained model
y_pred = pipeline.predict(X_test)
y_pred = pd.DataFrame(y_pred, columns=y_test.columns)

print(classification_report(y_test.values, y_pred.values, target_names=y_test.columns.values))

                        precision    recall  f1-score   support

               related       0.84      0.84      0.84      4037
               request       0.55      0.56      0.56       877
                 offer       0.05      0.05      0.05        21
           aid_related       0.63      0.61      0.62      2158
          medical_help       0.37      0.30      0.33       432
      medical_products       0.35      0.36      0.36       252
     search_and_rescue       0.32      0.23      0.27       171
              security       0.06      0.05      0.05       106
              military       0.38      0.38      0.38       172
           child_alone       0.00      0.00      0.00         0
                 water       0.65      0.63      0.64       320
                  food       0.70      0.72      0.71       574
               shelter       0.59      0.59      0.59       442
              clothing       0.52      0.45      0.48        84
                 money       0.34      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [9]:
# Alternative:
for col in y_test.columns:
    print(col)
    print(classification_report(y_test.loc[:,col].values, y_pred.loc[:,col].values))

related
             precision    recall  f1-score   support

          0       0.46      0.46      0.46      1207
          1       0.84      0.84      0.84      4037

avg / total       0.75      0.75      0.75      5244

request
             precision    recall  f1-score   support

          0       0.91      0.91      0.91      4367
          1       0.55      0.56      0.56       877

avg / total       0.85      0.85      0.85      5244

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5223
          1       0.05      0.05      0.05        21

avg / total       0.99      0.99      0.99      5244

aid_related
             precision    recall  f1-score   support

          0       0.73      0.75      0.74      3086
          1       0.63      0.61      0.62      2158

avg / total       0.69      0.69      0.69      5244

medical_help
             precision    recall  f1-score   support

          0       0.94      0.95      0

### 6. Improve your model
Use grid search to find better parameters. 

In [5]:
# List all parameters from pipeline
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f133563c620>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
               max_features=None, max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, presort=False, random_state=None,
               splitter='best'),
              n_jobs=1))],
 

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=21)

In [12]:
'''
Crucial parameters https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html 
to avoid fully grown and unpruned trees in decision tree classifier algorithms:
- clf__estimator__max_depth
- clf__estimator__min_samples_leaf
'''
parameters = {'clf__estimator__max_depth': [5,10,20],
                'clf__estimator__min_samples_leaf': [10,20,50]}

cv = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=3)
cv.fit(X_train, y_train)
cv.best_params_

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] clf__estimator__max_depth=5, clf__estimator__min_samples_leaf=10 
[CV]  clf__estimator__max_depth=5, clf__estimator__min_samples_leaf=10, score=0.22929480760978402, total=  12.7s
[CV] clf__estimator__max_depth=5, clf__estimator__min_samples_leaf=10 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   15.5s remaining:    0.0s


[CV]  clf__estimator__max_depth=5, clf__estimator__min_samples_leaf=10, score=0.2268631097124875, total=  12.6s
[CV] clf__estimator__max_depth=5, clf__estimator__min_samples_leaf=10 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:   31.0s remaining:    0.0s


[CV]  clf__estimator__max_depth=5, clf__estimator__min_samples_leaf=10, score=0.2369098712446352, total=  12.5s
[CV] clf__estimator__max_depth=5, clf__estimator__min_samples_leaf=20 
[CV]  clf__estimator__max_depth=5, clf__estimator__min_samples_leaf=20, score=0.22815047918752682, total=  12.1s
[CV] clf__estimator__max_depth=5, clf__estimator__min_samples_leaf=20 
[CV]  clf__estimator__max_depth=5, clf__estimator__min_samples_leaf=20, score=0.2260048633957946, total=  12.2s
[CV] clf__estimator__max_depth=5, clf__estimator__min_samples_leaf=20 
[CV]  clf__estimator__max_depth=5, clf__estimator__min_samples_leaf=20, score=0.23762517882689557, total=  12.2s
[CV] clf__estimator__max_depth=5, clf__estimator__min_samples_leaf=50 
[CV]  clf__estimator__max_depth=5, clf__estimator__min_samples_leaf=50, score=0.23415820340437707, total=  11.8s
[CV] clf__estimator__max_depth=5, clf__estimator__min_samples_leaf=50 
[CV]  clf__estimator__max_depth=5, clf__estimator__min_samples_leaf=50, score=0.22

[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed: 11.1min finished


{'clf__estimator__max_depth': 10, 'clf__estimator__min_samples_leaf': 50}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
cv.best_params_

{'clf__estimator__max_depth': 10, 'clf__estimator__min_samples_leaf': 50}

In [14]:
# Infer test data with trained model
y_pred = cv.predict(X_test)
y_pred = pd.DataFrame(y_pred, columns=y_test.columns)

print(classification_report(y_test.values, y_pred.values, target_names=y_test.columns.values))

                        precision    recall  f1-score   support

               related       0.82      0.93      0.87      4037
               request       0.70      0.52      0.60       877
                 offer       0.00      0.00      0.00        21
           aid_related       0.72      0.50      0.59      2158
          medical_help       0.56      0.27      0.37       432
      medical_products       0.74      0.31      0.44       252
     search_and_rescue       0.70      0.12      0.21       171
              security       0.00      0.00      0.00       106
              military       0.47      0.25      0.33       172
           child_alone       0.00      0.00      0.00         0
                 water       0.72      0.61      0.66       320
                  food       0.78      0.72      0.75       574
               shelter       0.76      0.58      0.66       442
              clothing       0.69      0.49      0.57        84
                 money       0.63      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [24]:
'''
Model to try: RandomForestClassifier (ensemble model)
Additional features: HashingVectorizer
'''
pipeline2 =  Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(RandomForestClassifier()))
                ])

In [25]:
pipeline2.fit(X_train, y_train)

# Infer test data with trained model
y_pred = pipeline2.predict(X_test)
y_pred = pd.DataFrame(y_pred, columns=y_test.columns)

print(classification_report(y_test.values, y_pred.values, target_names=y_test.columns.values))

                        precision    recall  f1-score   support

               related       0.84      0.92      0.88      4037
               request       0.84      0.37      0.51       877
                 offer       0.00      0.00      0.00        21
           aid_related       0.74      0.51      0.60      2158
          medical_help       0.56      0.06      0.11       432
      medical_products       0.67      0.10      0.18       252
     search_and_rescue       0.80      0.02      0.05       171
              security       0.00      0.00      0.00       106
              military       0.67      0.07      0.13       172
           child_alone       0.00      0.00      0.00         0
                 water       0.90      0.23      0.36       320
                  food       0.89      0.38      0.53       574
               shelter       0.83      0.23      0.36       442
              clothing       0.74      0.20      0.32        84
                 money       0.57      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

In [27]:
joblib.dump(cv.best_estimator_, 'classifier.pkl')

['classifier.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.